In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\Pedro_Carvalho1\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [3]:
#!connect mssql --kernel-name sqlstudy --connection-string @connStr

Kernel added: #!sql-sqlstudy

In [5]:
#!sql-sqlstudy
-- Criar tabela Signups
CREATE TABLE dbo.Signups (
    user_id INT NOT NULL PRIMARY KEY,
    time_stamp DATETIME NOT NULL
);

-- Criar tabela Confirmations
CREATE TABLE dbo.Confirmations (
    user_id INT NOT NULL,
    time_stamp DATETIME NOT NULL,
    action VARCHAR(20) NOT NULL CHECK (action IN ('confirmed', 'timeout')),
    CONSTRAINT PK_Confirmations PRIMARY KEY (user_id, time_stamp),
    CONSTRAINT FK_Confirmations_Signups FOREIGN KEY (user_id)
        REFERENCES dbo.Signups (user_id)
);



Commands completed successfully.

In [6]:
#!sql-sqlstudy
-- Inserir dados na tabela Signups
INSERT INTO dbo.Signups (user_id, time_stamp)
VALUES
(3, '2020-03-21 10:16:13'),
(7, '2020-01-04 13:57:59'),
(2, '2020-07-29 23:09:44'),
(6, '2020-12-09 10:39:37');

-- Inserir dados na tabela Confirmations
INSERT INTO dbo.Confirmations (user_id, time_stamp, action)
VALUES
(3, '2021-01-06 03:30:46', 'timeout'),
(3, '2021-07-14 14:00:00', 'timeout'),
(7, '2021-06-12 11:57:29', 'confirmed'),
(7, '2021-06-13 12:58:28', 'confirmed'),
(7, '2021-06-14 13:59:27', 'confirmed'),
(2, '2021-01-22 00:00:00', 'confirmed'),
(2, '2021-02-28 23:59:59', 'timeout');


(4 rows affected)

(7 rows affected)

In [7]:
#!sql-sqlstudy
SELECT * FROM dbo.Signups;
SELECT * FROM dbo.Confirmations;

(4 rows affected)

(7 rows affected)

user_id,time_stamp
2,2020-07-29 23:09:44Z
3,2020-03-21 10:16:13Z
6,2020-12-09 10:39:37Z
7,2020-01-04 13:57:59Z


user_id,time_stamp,action
2,2021-01-22 00:00:00Z,confirmed
2,2021-02-28 23:59:59Z,timeout
3,2021-01-06 03:30:46Z,timeout
3,2021-07-14 14:00:00Z,timeout
7,2021-06-12 11:57:29Z,confirmed
7,2021-06-13 12:58:28Z,confirmed
7,2021-06-14 13:59:27Z,confirmed


In [ ]:
The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. 
The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.
Return the result table in any order.
The result format is in the following example.


Example 1:

Input: 
Signups table:
+---------+---------------------+
| user_id | time_stamp          |
+---------+---------------------+
| 3       | 2020-03-21 10:16:13 |
| 7       | 2020-01-04 13:57:59 |
| 2       | 2020-07-29 23:09:44 |
| 6       | 2020-12-09 10:39:37 |
+---------+---------------------+
Confirmations table:
+---------+---------------------+-----------+
| user_id | time_stamp          | action    |
+---------+---------------------+-----------+
| 3       | 2021-01-06 03:30:46 | timeout   |
| 3       | 2021-07-14 14:00:00 | timeout   |
| 7       | 2021-06-12 11:57:29 | confirmed |
| 7       | 2021-06-13 12:58:28 | confirmed |
| 7       | 2021-06-14 13:59:27 | confirmed |
| 2       | 2021-01-22 00:00:00 | confirmed |
| 2       | 2021-02-28 23:59:59 | timeout   |
+---------+---------------------+-----------+
Output: 
+---------+-------------------+
| user_id | confirmation_rate |
+---------+-------------------+
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |
+---------+-------------------+
Explanation: 
User 6 did not request any confirmation messages. The confirmation rate is 0.
User 3 made 2 requests and both timed out. The confirmation rate is 0.
User 7 made 3 requests and all were confirmed. The confirmation rate is 1.
User 2 made 2 requests where one was confirmed and the other timed out. The confirmation rate is 1 / 2 = 0.5.


In [48]:
/* Write your T-SQL query statement below */
#!sql-sqlstudy
select
    s.user_id
    -- ,c.user_id
    -- ,c.time_stamp
    -- ,c.action
    -- ,SUM( case when c.action = 'confirmed'  then 1 else 0 end ) as count_confirmed
    -- ,COUNT( c.action ) as count_total
    , CASE
        WHEN 
            SUM( case when c.action = 'confirmed'  then 1 else 0 end ) * 1.0
            / NULLIF( COUNT( c.action ) , 0)
            IS NULL THEN 0
        ELSE 
            CAST(
                ROUND( 
                    SUM( case when c.action = 'confirmed'  then 1 else 0 end ) * 1.0
                    / NULLIF( COUNT( c.action ) , 0)
                , 2 ) as DECIMAL(10,2) )
    END  as confirmation_rate  
from signups s
left join confirmations c on s.user_id = c.user_id
group by s.user_id

Commands completed successfully.

(4 rows affected)

user_id,confirmation_rate
2,0.50
3,0.00
6,0.00
7,1.00


In [ ]:
/* Write your T-SQL query statement below */
#!sql-sqlstudy
select
    s.user_id
    ,CAST( ROUND( AVG( IIF(c.action = 'confirmed',1,0) * 1.0 ), 2) AS DECIMAL(10,2)) AS confirmation_rate 
from signups s
left join confirmations c on s.user_id = c.user_id
group by s.user_id

(4 rows affected)

user_id,TEST
2,0.50
3,0.00
6,0.00
7,1.00
